# Python Implementation of the Sigma Protocol

This notebook presents a detailed Python implementation of the Sigma Protocol, a fundamental zero-knowledge proof system in cryptography. It serves as a practical complement to the theoretical discussion provided in [Intro to Sigma Protocols](./sigma_1.ipynb), where the mathematical foundations of the protocol are explored.

---

## Setup

The setup involves defining functions for parameter generation and key creation. These functions are implemented manually in this notebook to provide a clear understanding of the protocol's inner workings.

In [1]:
import random

def generate_parameters():
    while True:
        # Generate a random prime p
        p = random.randint(20, 100)
        if all(p % i != 0 for i in range(2, int(p**0.5) + 1)):
            # Find a prime q such that q divides p-1
            for q in range(2, p):
                if (p - 1) % q == 0 and all(q % i != 0 for i in range(2, int(q**0.5) + 1)):
                    # Find a generator g for the subgroup of order q
                    for g in range(2, p):
                        if pow(g, q, p) == 1 and pow(g, (p-1)//q, p) != 1:
                            return p, q, g

def keygen(p, q, g):
    x = random.randint(1, q-1)  # Prover's secret
    y = pow(g, x, p)            # Public key
    return x, y

## Protocol Execution

The following code snippets illustrate the process of proving the Schnorr Identification Protocol, the response and the verification steps.

In [2]:
def sigma_commit(p, q, g):
    r = random.randint(0, q-1)
    t = pow(g, r, p)
    return r, t

def sigma_response(q, x, r, c):
    s = (r + c * x) % q
    return s

def sigma_verify(p, g, y, t, c, s):
    lhs = pow(g, s, p)
    rhs = (t * pow(y, c, p)) % p
    return lhs == rhs

## Worked Example

Let's work through a full protocol instance using small primes for easy verification.

In [3]:
p, q, g = generate_parameters()
print(f"Generated parameters: p = {p}, q = {q}, g = {g}")
x, y = keygen(p, q, g)

print(f"Public parameters: p = {p}, q = {q}, g = {g}")
print(f"Prover's secret key: x = {x}")
print(f"Prover's public key: y = {y}")
print()

Generated parameters: p = 79, q = 2, g = 78
Public parameters: p = 79, q = 2, g = 78
Prover's secret key: x = 1
Prover's public key: y = 78



### Example 1: Single protocol execution

The honest prover successfully demonstrates knowledge of the secret `x`. The verifier accepts the proof, confirming the protocol's correctness.

In [9]:
print("=== Example 1: Single Protocol Execution ===")
# Constructing the Sigma Protocol
r, t = sigma_commit(p, q, g)
print(f"Prover commits with random r = {r} to produce t = {t}")

# Adding a challenge
c = random.randint(0, q-1)
print(f"Verifier selects random challenge c = {c}")

# Sigma response
s = sigma_response(q, x, r, c)
print(f"Prover responds with s = {s}")

# Verification
result = sigma_verify(p, g, y, t, c, s)
print(f"Verifier checks and result is: {result}")

=== Example 1: Single Protocol Execution ===
Prover commits with random r = 0 to produce t = 1
Verifier selects random challenge c = 1
Prover responds with s = 1
Verifier checks and result is: True


### Example 2: Multiple protocol executions with different challenges

The protocol is executed multiple times with different challenges. Each execution demonstrates the repeatability and robustness of the protocol.

In [13]:
print("=== Example 2: Multiple Protocol Executions ===")
for i in range(3):
    print(f"\n--- Execution {i+1} ---")
    
    # Prover computes commitment
    r, t = sigma_commit(p, q, g)
    print(f"Prover commits with random r = {r} to produce t = {t}")

    # Verifier sends challenge
    c = random.randint(0, q-1)
    print(f"Verifier selects challenge c = {c}")

    # Prover computes response
    s = sigma_response(q, x, r, c)
    print(f"Prover responds with s = {s}")

    # Verifier checks
    result = sigma_verify(p, g, y, t, c, s)
    print(f"Verification result: {result}")

=== Example 2: Multiple Protocol Executions ===

--- Execution 1 ---
Prover commits with random r = 1 to produce t = 78
Verifier selects challenge c = 1
Prover responds with s = 0
Verification result: True

--- Execution 2 ---
Prover commits with random r = 0 to produce t = 1
Verifier selects challenge c = 1
Prover responds with s = 1
Verification result: True

--- Execution 3 ---
Prover commits with random r = 0 to produce t = 1
Verifier selects challenge c = 0
Prover responds with s = 0
Verification result: True


### Example 3: Simulating a cheating prover

The cheating prover attempts to convince the verifier without knowing the secret. The verifier rejects the proof, demonstrating the protocol's soundness.

In [8]:
print("=== Example 3: Simulating a Cheating Prover ===")
# Cheating prover does not know x but tries to fake a response
fake_r = random.randint(0, q-1)
fake_t = pow(g, fake_r, p)
print(f"Cheating prover sends fake commitment fake_t = {fake_t}")

# Verifier sends challenge
c = random.randint(0, q-1)
print(f"Verifier selects challenge c = {c}")

# Cheating prover tries to fake a response without knowing x
fake_s = random.randint(0, q-1)
print(f"Cheating prover sends fake response fake_s = {fake_s}")

# Verifier checks
result = sigma_verify(p, g, y, fake_t, c, fake_s)
print(f"Verification result (expected to fail): {result}")

=== Example 3: Simulating a Cheating Prover ===
Cheating prover sends fake commitment fake_t = 78
Verifier selects challenge c = 0
Cheating prover sends fake response fake_s = 0
Verification result (expected to fail): False
